In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, LeakyReLU
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger

import os

# === Step 1: Load and preprocess ===
df = pd.read_csv("acquiredDataset.csv")
df.head()


,attention,meditation,delta,theta,lowAlpha,highAlpha,lowBeta,highBeta,lowGamma,highGamma,classification
0,26,34,960462,277180,26575,27356,26575,13056,9126,2683,0
1,29,54,39145,28225,20172,39551,20172,9933,5237,4750,0
2,40,48,75410,43144,8601,13564,8601,11663,2515,3251,0
3,66,47,16057,41211,2534,34254,2534,27663,11396,2825,0
4,81,67,10304,47239,33158,47349,33158,16328,5298,5471,0


In [4]:

# Compute band features
df['alpha'] = (df['lowAlpha'] + df['highAlpha']) / 2
df['beta'] = (df['lowBeta'] + df['highBeta']) / 2
df = df[['alpha', 'beta', 'theta', 'delta', 'classification']]
df.head()

,alpha,beta,theta,delta,classification
0,26965.5,19815.5,277180,960462,0
1,29861.5,15052.5,28225,39145,0
2,11082.5,10132.0,43144,75410,0
3,18394.0,15098.5,41211,16057,0
4,40253.5,24743.0,47239,10304,0


In [5]:


# === Step 2: Log-transform & MinMax scale ===
features = df.drop(columns=['classification'])
labels = df['classification']

# Apply np.log1p
features_log = np.log1p(features)

# MinMax scaling
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features_log)

# Combine with label
df_processed = pd.DataFrame(features_scaled, columns=features.columns)
df_processed['classification'] = labels.values


In [6]:

# === Step 3: Generate sequences ===
def generate_sequences(dataframe, seq_length):
    sequences, labels = [], []
    for i in range(len(dataframe) - seq_length):
        seq = dataframe.iloc[i:i+seq_length, :-1].values
        label = dataframe.iloc[i+seq_length-1, -1]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

SEQ_LEN = 20
X, y = generate_sequences(df_processed, SEQ_LEN)

# === Step 4: Split into train/val/test (60/20/20) ===
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42, stratify=y_temp) 


In [7]:


# === Step 5: Model definition ===
def build_advanced_lstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(128, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(64, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(32, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_advanced_lstm_model((X_train.shape[1], X_train.shape[2]))


/home/purab/.local/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:


# === Step 6: Callbacks ===
def create_callbacks():
    os.makedirs("models", exist_ok=True)
    return [
        EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True),
        ModelCheckpoint("models/best_classifier.keras", save_best_only=True, monitor='val_accuracy', mode='max'),
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=1e-6, verbose=1),
        TensorBoard(log_dir='logs'),
        CSVLogger("training_log.csv")
    ]
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 20, 256)        │       136,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 20, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 329,857 (1.26 MB)

 Trainable params: 328,641 (1.25 MB)

 Non-trainable params: 1,216 (4.75 KB)

In [ ]:

# === Step 7: Train the model ===
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=create_callbacks(),
    verbose=1
)


In [61]:

y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()

print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step

Test Accuracy: 0.8842530282637954
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.92      0.90       423
           1       0.89      0.84      0.86       320

    accuracy                           0.88       743
   macro avg       0.88      0.88      0.88       743
weighted avg       0.88      0.88      0.88       743

